
# RNN em Física: Séries temporais (manchas solares) usando RNN
## Alexandre Suaide

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Input
from keras.optimizers import Adam

from sklearn.preprocessing import MinMaxScaler


In [ ]:
# Funções necessárias

def get_XY(data, time_steps):
    Y_ind = np.arange(time_steps, len(data), time_steps)
    Y = data[Y_ind]
    rows_x = len(Y)
    X = data[range(time_steps*rows_x)]
    X = np.reshape(X, (rows_x, time_steps, 1))    
    return X, Y

In [ ]:
# Carrega o dataset

df = pd.read_csv("../data/manchas-solares.csv")
df.head()

In [ ]:
# Converte em uma sequência

data = np.array(df['Sunspots'].values.astype('float32')).reshape(-1, 1)
data

In [ ]:
# Escalona os dados entre 0 e 1

scaler = MinMaxScaler(feature_range=(0, 1))
data = scaler.fit_transform(data)
data.shape

In [ ]:
# Divide a sequência entre treino e predição

fraction = 0.6
n = len(data)
split = int(fraction * n) # 80% train, 20% test
train = data[range(split)]
test = data[split:]

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
# Divide a sequência em subconjuntos de steps

steps = 12
train_x, train_y = get_XY(train,steps)
test_x, test_y = get_XY(test,steps)


In [ ]:
# Construção do modelo no Keras

model = Sequential()
model.add(Input(shape=train.shape))
model.add(SimpleRNN(units = 3, activation = 'tanh'))
model.add(Dense(units = 1, activation = 'tanh'))
        
optimizer = Adam()

model.compile(optimizer = optimizer , loss = "MSE", metrics=["accuracy"])
model.summary()

In [ ]:
# Treina a rede

epochs = 30
batch_size = 1
history = model.fit(train_x, train_y, epochs = epochs, batch_size = batch_size)

In [ ]:
# faz figura da perda e da acurácia para cada época

plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(history.history['loss'], label='treino')
plt.title('Perda')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['accuracy'], label='treino')
plt.title('Acurácia')
plt.legend()
plt.show()

In [ ]:
# Realiza as previsões, tanto para o período de treino quando para o período de teste

train_predict = model.predict(train_x)
test_predict = model.predict(test_x)

In [ ]:
full_y = np.append(train_y, test_y)
full_p = np.append(train_predict, test_predict)

In [ ]:
# Faz a figura

plt.figure()
plt.plot(full_y)
plt.plot(full_p)
plt.axvline(x=len(train_y), color='r')
plt.legend(['Dados', 'Predições'])

In [ ]:
# salva o modelo para uso futuro em novos dados. Dai não precisa treinar a rede novamente

model.save("manchas_solares.keras")
print("Modelo gravado como manchas_solares.keras")